###Set working directory

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%cd /content/drive/My Drive/Live_Projects/
# !ls

/content/drive/My Drive/Live_Projects


###Import essential libraries

In [100]:
from PIL import Image
import os
import re

import matplotlib.pyplot as plt
import numpy as np

import pylab as pl
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.svm import SVC

from skimage.feature import hog

###Obtain image feature & label

In [101]:
folder='a1_TrainingPhotos/' # location of training images
images = []
i=0

X=[]
Y=[]

for filename in os.listdir(folder):   # run the loop for all files in training folder
    img = Image.open(os.path.join(folder,filename)) # open image 

    img=img.resize([32,64])  # resize image  to a fixed size
    
    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2, 2), visualize=True, multichannel=True) # obtain HOG features
    
    X.append(fd)

    # obtain training labels from file name
    
    if re.match("Warnar*", filename):
      Yt=0
    elif re.match("Elon*", filename):
      Yt=1
    elif re.match("Messi*", filename):
      Yt=2
    elif re.match("Virat*", filename):
      Yt=3
    elif re.match("Obama*", filename):
      Yt=4
    else:
      Yt=5

    Y.append(Yt)
    
    i=i+1


<ipython-input-101-65747da7fa20>:13: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(img, orientations=9, pixels_per_cell=(4,4), cells_per_block=(2, 2), visualize=True, multichannel=True) # obtain HOG features


In [102]:
fd.shape

(3780,)

In [103]:
target_names=['Elon', 'Virat', 'Messi', 'Warnar', 'Obama']
# 'AamirKhan', 'JayaBadhuri', 'HritikRoshan', 'AnupamKher', 

###Split data into training/test

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###Reduce dimenions with PCA

In [105]:
n_components = 25
pca = PCA(n_components=n_components, whiten=True).fit(X_train)

In [106]:
# project training and test data to lower dimension 
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [107]:
# Save PCA coefficients for use at prediction

import pickle as pk
pk.dump(pca, open("c1_FR_PCA.pkl","wb"))

###Train SVM classifier

In [108]:
print("Fitting the classifier to the training set")
param_grid = {
         'C': [1e3, 5e3, 1e4, 5e4, 1e5],
          'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
          }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid)
clf = clf.fit(X_train_pca, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)

Fitting the classifier to the training set


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.0001, probability=True)


In [109]:
# Save best SVM classifier weights for use at prediction

import joblib
joblib.dump(clf.best_estimator_, 'c2_FR_svm_classifier.pkl', compress = 1) # Only best parameters

['c2_FR_svm_classifier.pkl']

###Model performance on test set

In [110]:
y_pred = clf.predict(X_test_pca)
y_pred

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5])

In [111]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.97      1.00      0.98        93

    accuracy                           0.97        96
   macro avg       0.32      0.33      0.33        96
weighted avg       0.94      0.97      0.95        96



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
